# 📊 Teoría de las Bandas de Bollinger

Las **Bandas de Bollinger**, desarrolladas por John Bollinger en los años 80, son un indicador técnico que mide la **volatilidad** del mercado y ayuda a identificar niveles de **sobrecompra** y **sobreventa**.

---

## 🔹 Componentes de las Bandas

1. **Media móvil simple (SMA):**  
   Generalmente de 20 períodos:  

   $$
   SMA_t = \frac{1}{n} \sum_{i=0}^{n-1} P_{t-i}
   $$

2. **Desviación estándar ($\sigma$):**  
   Mide la volatilidad de los precios respecto a la media:  

   $$
   \sigma_t = \sqrt{\frac{1}{n} \sum_{i=0}^{n-1} (P_{t-i} - SMA_t)^2}
   $$

3. **Bandas superior e inferior:**  

   - Banda Superior:  
   $$
   BB^{sup}_t = SMA_t + k \cdot \sigma_t
   $$

   - Banda Inferior:  
   $$
   BB^{inf}_t = SMA_t - k \cdot \sigma_t
   $$

   Donde:  
   - $n$ = número de períodos (usualmente 20).  
   - $k$ = multiplicador de la desviación estándar (usualmente 2).  

---

## 🔹 Interpretación

- Cuando el precio se acerca a la **banda superior** → condición de **sobrecompra**.  
- Cuando el precio se acerca a la **banda inferior** → condición de **sobreventa**.  
- El **estrechamiento** de las bandas indica baja volatilidad (posible ruptura).  
- La **expansión** de las bandas indica alta volatilidad.  

---

## 🔹 Señales comunes

1. **Toque de banda:**
   - Si el precio toca la banda superior → posible corrección bajista.  
   - Si el precio toca la banda inferior → posible rebote alcista.  

2. **Bollinger Squeeze:**  
   - Cuando las bandas se estrechan mucho → posible movimiento fuerte (breakout).  

3. **Confirmación de tendencia:**  
   - Precios que se mantienen pegados a la banda superior → tendencia alcista fuerte.  
   - Precios que se mantienen pegados a la banda inferior → tendencia bajista fuerte.  

---

## 📊 Parámetros típicos

- Período: $n = 20$  
- Desviación estándar: $k = 2$  

Se expresan como **BB(20,2)**.

---

In [1]:
# !pip install yfinance plotly

import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

# ------------------ Bollinger Bands ------------------
def compute_bollinger(df: pd.DataFrame, period: int = 20, num_std: int = 2):
    sma = df['Close'].rolling(period).mean()
    std = df['Close'].rolling(period).std()
    upper = sma + num_std * std
    lower = sma - num_std * std
    return sma, upper, lower

# ------------------ Gráfico Velas + Bollinger ------------------
def plot_candles_bollinger(
    ticker: str,
    start_date: str,
    end_date: str,
    period: int = 20,
    num_std: int = 2
):
    df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False, progress=False)
    if df.empty:
        raise ValueError(f"Sin datos para {ticker} en {start_date} → {end_date}")

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df[['Open','High','Low','Close']].dropna().copy()

    # Calcular Bollinger Bands
    sma, upper, lower = compute_bollinger(df, period=period, num_std=num_std)

    fig = go.Figure()

    # --- Velas (verde/rojo) ---
    fig.add_trace(go.Candlestick(
        x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'],
        increasing_line_color='green', increasing_fillcolor='green',
        decreasing_line_color='red', decreasing_fillcolor='red',
        name=ticker
    ))

    # --- Banda superior (línea goldenrod) ---
    fig.add_trace(go.Scatter(
        x=df.index, y=upper,
        mode="lines",
        line=dict(color="goldenrod", width=1),
        name="Upper Band"
    ))

    # --- Banda inferior (línea goldenrod + sombra azul claro hasta la superior) ---
    fig.add_trace(go.Scatter(
        x=df.index, y=lower,
        mode="lines",
        line=dict(color="goldenrod", width=1),
        name="Lower Band",
        fill='tonexty', fillcolor='rgba(30, 144, 255, 0.15)'  # azul claro
    ))

    # --- Línea SMA20 (azul) ---
    fig.add_trace(go.Scatter(
        x=df.index, y=sma,
        mode="lines", line=dict(color="blue", width=2),
        name=f"SMA{period}"
    ))

    # Layout
    fig.update_layout(
        title=f"{ticker} | Velas + Bandas de Bollinger ({period}, {num_std}σ)",
        xaxis_rangeslider_visible=False,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(l=90, r=90, t=60, b=40)
    )

    fig.update_yaxes(title_text="Precio")
    fig.show()




In [2]:
#@title Seleccione Acción y Fecha {run:'auto'}
ticker = "NVDA" #@param ["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "TSM","BRK.B", "V", "JPM", "XOM", "LLY", "MRK", "UNH", "PG", "MA","CVX", "KO", "PEP", "COST", "TMO", "ORCL", "CSCO", "NKE", "VZ", "ASML", "TXN", "ABT", "TM", "SAP", "AMD", "NFLX", "NOW", "ADBE", "LVMUY", "BABA", "SHEL", "TMUS", "QCOM", "PFE", "SNY", "AZN", "TOT", "GSK", "RIO", "BHP", "MCD​"]
#ticker = "BTC-USD" #@param ["BTC-USD", "ETH-USD", "USDT-USD", "XRP-USD", "LTC-USD", "ADA-USD", "DOT-USD", "BCH-USD", "XLM-USD", "LINK-USD"]

start_date = '2024-09-09' #@param {type:'date'}
end_date = '2025-09-09' #@param {type:'date'}

In [3]:
plot_candles_bollinger(
    ticker, start_date, end_date,
    period=20, num_std=2
)

In [4]:
3# Cómo calcular las ganancias con esta estrategia
inversion_inicial = 1_000_000.00 # usd
# Compras
# 1
lower1 = 172.42
acciones1 = inversion_inicial / lower1
upper1 = 253.48
ganancia1 = acciones1 * upper1
#
print(f"Inversion Inicial {inversion_inicial:,.2f}")
print(f"Acciones Compradas {acciones1:,.2f}")
print(f"Sobre Venta {lower1:,.2f}")
print(f"Sobre Compra {upper1:,.2f}")
print(f"Ganancia {ganancia1:,.2f}")

Inversion Inicial 1,000,000.00
Acciones Compradas 5,799.79
Sobre Venta 172.42
Sobre Compra 253.48
Ganancia 1,470,131.08


In [5]:
# Reporte Final
rendimiento = (ganancia1 - inversion_inicial) / inversion_inicial
print(f"Ganancia Final {ganancia1:,.2f}")
print(f"Rendimiento {rendimiento:,.2%}")

Ganancia Final 1,470,131.08
Rendimiento 47.01%
